In [29]:
import ScraperFC as sfc
import traceback
import pandas as pd
pd.options.display.max_columns = 50
from IPython.display import clear_output

In [30]:
# # scraper = sfc.Understat()
# scraper = sfc.FBRef()

# error = False
# shapes = []
# for yr in range(2018,2021): #1988,2021):
#     for lg in ['EPL','La Liga','Bundesliga','Serie A','Ligue 1']:

#         try:
#         #     out = scraper.scrape_situations(2019, "La Liga")
#         #     out = scraper.scrape_formations(2020, "EPL")
#         #     out = scraper.scrape_game_states(2020, "EPL")

#             out = scraper.scrape_matches(yr, lg)
#             shapes.append(
#                 [lg, yr, out.shape, out["Home Player Stats"], out["Away Player Stats"]]
#             )
#     #         scraper.close()
#         except:  
#             traceback.print_exc()
#             scraper.close()
#             error = True
#             break

#     if error:
#         break
#     scraper.close()

In [31]:
scraper = sfc.FBRef()

try:
    out = scraper.scrape_matches(2021, "La Liga")
    scraper.close()
except:  
    traceback.print_exc()
    scraper.close()

In [32]:
out.shape

(380, 23)

In [33]:
out["Home Player Stats"]

0      Team Sheet       Manchester Utd (4-3-3) Manche...
1      Team Sheet       Newcastle Utd (4-4-1-1) Newca...
2      Team Sheet       Watford (4-2-2-2) Watford (4-...
3      Team Sheet       Huddersfield (3-5-1-1) Hudder...
4      Team Sheet       Bournemouth (4-4-2) Bournemou...
                             ...                        
375    Team Sheet       Fulham (4-2-3-1)          Ful...
376    Team Sheet       Leicester City (4-1-4-1) Leic...
377    Team Sheet       Burnley (4-4-2)        Burnle...
378    Team Sheet       Tottenham (4-2-3-1◆) Tottenha...
379    Team Sheet       Watford (4-4-2)   Watford (4-...
Name: Home Player Stats, Length: 380, dtype: object

In [34]:
out["Away Player Stats"]

0      Team Sheet       Leicester City (4-2-3-1) Leic...
1      Team Sheet       Tottenham (4-2-3-1) Tottenham...
2      Team Sheet       Brighton (4-4-1-1) Brighton (...
3      Team Sheet       Chelsea (4-3-3)    Chelsea (4...
4      Team Sheet       Cardiff City (4-1-4-1) Cardif...
                             ...                        
375    Team Sheet       Newcastle Utd (3-4-3) Newcast...
376    Team Sheet       Chelsea (4-3-3)    Chelsea (4...
377    Team Sheet       Arsenal (4-2-3-1)        Arse...
378    Team Sheet       Everton (4-2-3-1)   Everton (...
379    Team Sheet       West Ham (4-2-3-1) West Ham (...
Name: Away Player Stats, Length: 380, dtype: object